In [38]:
import os
import pandas as pd
import requests

from Bio import AlignIO

In [11]:
path = os.getcwd()

In [102]:
var_dict = {
'H1.0':'H1.0',
'H1.1':'H1.1',#new
'H1.2':'H1.2',#new
'H1.3':'H1.3',#new
'H1.4':'H1.4',#new
'H1.5':'H1.5',#new
'TS H1.6':'TS_H1.6',
'TS H1.7':'TS_H1.7',
'OO H1.8':'OO_H1.8',
'TS H1.9':'TS_H1.9',
'H1.10':'H1.10',
'TS H2A.1':'H2A.1',
'canonical H2A':'canonical_H2A',
'H2A.J(?)':'H2A.J',#new
'H2A.X':'H2A.X',
'H2A.Z.1':'H2A.Z',
'H2A.Z.2':'H2A.Z',
'macroH2A.1':'macroH2A',
'macroH2A.2':'macroH2A',
'H2A.B.1':'H2A.B',
'H2A.B.2':'H2A.B',
'H2A.P':'H2A.P',
'TS H2B.1':'H2B.1', 
'canonical H2B':'canonical_H2B',
'H2B.S(?)':'H2B.S',#new
'H2B.W':'H2B.W',
'?':'skip', #new, to change (add histone_type to variant name)
'canonical H3.1':'canonical_H3',
'canonical H3.2':'canonical_H3',
'H3.Y.1':'H3.Y',
'H3.Y.2':'H3.Y',
'canonical H3(?)':'canonical_H3',
'H3.3':'H3.3',
'TS H3.4':'TS_H3.4',
'H3.5':'H3.5',
'cenH3':'cenH3',
'canonical H4':'canonical_H4',
}

var_dict_reverse = {v: k for k, v in var_dict.items()}

In [62]:
hist_prot = pd.read_csv(os.path.join(path, "static/human_hist/info/", "human_hist_proteins.csv"))
hist_genes = pd.read_csv(os.path.join(path, "static/human_hist/info/", "human_hist_genes.csv"), usecols=['Histone type', 'Histone variant', 'HGNC Symbol'])

hist_prot = pd.merge(hist_prot, hist_genes.drop_duplicates(), on='HGNC Symbol', how='left')

In [107]:
%%time

seqs=[]

for i in hist_prot['Protein stable ID']:
    seq=requests.get('http://rest.ensembl.org/sequence/id/%s?content-type=text/plain'%i).content
    seqs.append(seq.decode("utf-8"))
    
hist_seq = dict(zip(hist_prot['RefSeq peptide ID'],seqs))
hist_prot['seq']=seqs

CPU times: user 349 ms, sys: 53.1 ms, total: 402 ms
Wall time: 21.8 s


In [134]:
len(hist_prot)

101

In [135]:
len(hist_seq)

97

In [137]:
hist_prot['RefSeq peptide ID'].value_counts()

NP_004884       2
NP_542160       2
NP_001299582    2
NP_001035248    2
NP_066390       1
               ..
NP_066409       1
NP_613075       1
NP_003486       1
NP_958844       1
NP_005313       1
Name: RefSeq peptide ID, Length: 97, dtype: int64

In [141]:
for i in ['NP_004884', 'NP_542160','NP_001299582','NP_001035248']:
    print( hist_prot['seq'].loc[hist_prot['RefSeq peptide ID'] == i].nunique() )

1
1
1
1


In [ ]:


# alignment = AlignIO.read(open("PF09395_seed.sth"), "stockholm")
# print("Alignment length %i" % alignment.get_alignment_length())
# for record in alignment:
#     print(record.seq + " " + record.id)

In [186]:
hist_type = 'H4'
hist_var = 'canonical_H4'


alignment = AlignIO.read(os.path.join(path,"static/browse/seeds/{}/".format(hist_type),"{}.fasta".format(hist_var)), "fasta")

In [187]:
fasta_variant = []
for record in alignment:
    fasta_variant.append(record.id.split('|')[1].split('.')[0])

var_list = hist_prot['RefSeq peptide ID'].loc[hist_prot['Histone variant']=='{}'.format(var_dict_reverse[hist_var])].values

uniq_seq = set()
for refseq in var_list:
    if refseq in fasta_variant:
        print(f'{refseq} in file')
    else:
        print(f'{refseq} not in file, added') 
        uniq_seq.add(hist_seq[refseq])
        #alignment.add_sequence(f'Homo|{refseq}|{hist_var}', hist_seq[refseq])

for i in uniq_seq:
    refseqs = list(hist_prot['RefSeq peptide ID'].loc[hist_prot['seq']==i])
    refseqs = ','.join(refseqs) 
    alignment.add_sequence(f'Homo|{refseqs}|{hist_var}', i)
        

NP_003529 not in file, added
NP_003535 not in file, added
NP_003533 not in file, added
NP_003530 not in file, added
NP_003536 not in file, added
NP_003531 not in file, added
NP_003538 in file
NP_003534 not in file, added
NP_003486 not in file, added
NP_068803 not in file, added
NP_003532 not in file, added
NP_003537 not in file, added
NP_003539 not in file, added
NP_001029249 in file
NP_778224 not in file, added


In [188]:
for r in alignment:
    print(r.id + "\n" + r.seq)

Arabidopsis|NP_180441.1|canonical_H4
MSGRGKGGK-GLGKGGAKRH-RKVLRDNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKIFLENVIRDAVTYTEHARRKTVTAMDVVYALKRQGRTLYGFGG
Caenorhabditis|NP_492641.1|canonical_H4
MSGRGKGGK-GLGKGGAKRH-RKVLRDNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKVFLENVIRDAVTYCEHAKRKTVTAMDVVYALKRQGRTLYGFGG
Drosophila|NP_001027352.1|canonical_H4
MTGRGKGGK-GLGKGGAKRH-RKVLRDNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKVFLENVIRDAVTYTEHAKRKTVTALDVVYALKRQGRTLYGFGG
Gallus|NP_001032934.1|canonical_H4
MSGRGKGGK-GLGKGGAKRH-RKVLRDNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKVFLENVIRDAVTYTEHAKRKTVTAMDVVYALKRQGRTLYGFGG
Heterocephalus|XP_012928609.1|canonical_H4
MSGRGKGGK-GLGKGGAKRH-RKVLRDNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKVFLENVIRDAVTYTEHAKRKTVTAMDVVYALKRQGRTLYGFGG
Homo|NP_003538.1|canonical_H4
MSVRGKAGK-GLGKGGAKCH-RKVLSDNIQGITKCTIRRLARHGGVKRILGLIYEETRRVFKVFLENVIWYAVTNTEHAKRKTVTAMAVVYVLKRQGRTL-----
Homo|NP_001029249.1|canonical_H4
MSGRGKGGK-GLGKGGAKRH-RKVLRDNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKVFLENVIRDAVTYTEHAKRKTVTAMDVVYALKRQGRTLYGFGG
M